In [41]:
import pandas as pd
data = pd.read_csv('data/ab_data_tourist.csv')
data.head()

,user_id,date,group,purchase,price
0,851104,2021-01-21,A,0,0
1,804228,2021-01-12,A,0,0
2,661590,2021-01-11,B,0,0
3,853541,2021-01-08,B,0,0
4,864975,2021-01-21,A,1,150000


## Анализ структуры и предобработка данных

### Информаця о структуре

In [42]:
print("\nИнформация о данных:")
print(data.info())


Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   294478 non-null  int64 
 1   date      294478 non-null  object
 2   group     294478 non-null  object
 3   purchase  294478 non-null  int64 
 4   price     294478 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 11.2+ MB
None


In [43]:
print("\nОсновные статистики:")
print(data.describe())


Основные статистики:
             user_id       purchase          price
count  294478.000000  294478.000000  294478.000000
mean   787974.124733       0.119659   10154.782361
std     91210.823776       0.324563   35153.824872
min    630000.000000       0.000000       0.000000
25%    709032.250000       0.000000       0.000000
50%    787933.500000       0.000000       0.000000
75%    866911.750000       0.000000       0.000000
max    945999.000000       1.000000  200000.000000


In [44]:
print("\nУникальные значения в столбце 'group':")
print(data['group'].value_counts())


Уникальные значения в столбце 'group':
group
B    147276
A    147202
Name: count, dtype: int64


### Преобразование к типу данных

In [45]:
#Преобразуем дату в datetime
data['date'] = pd.to_datetime(data['date'])
#Преобразуем group в категориальный тип
data['group'] = data['group'].astype('category')
#Преобразуем purchase в булевый тип (0/1)
data['purchase'] = data['purchase'].astype(bool)

In [46]:
print("Типы данных после преобразования:")
print(data.dtypes)

Типы данных после преобразования:
user_id              int64
date        datetime64[ns]
group             category
purchase              bool
price                int64
dtype: object


### Длительность a/b тестирования

In [47]:
#Находим минимальную и максимальную дату для каждой группы
date_range_by_group = data.groupby('group')['date'].agg(['min', 'max'])
date_range_by_group['duration'] = (date_range_by_group['max'] - date_range_by_group['min']).dt.days

In [48]:
print("Диапазон дат по группам:")
print(date_range_by_group)
#Если длительности отличаются, можно обрезать данные до общего периода
if len(date_range_by_group['duration'].unique()) > 1:
    print("Длительности тестирования различаются между группами!")
    common_start = date_range_by_group['min'].max()
    common_end = date_range_by_group['max'].min()
    print(f"Общий период тестирования: с {common_start} по {common_end}")
    #Обрезаем данные до общего периода
    data = data[(data['date'] >= common_start) & (data['date'] <= common_end)]
    print(f"Размер данных после обрезки: {data.shape}")
else:
    print("Длительности тестирования одинаковы для всех групп!")

Диапазон дат по группам:
             min        max  duration
group                                
A     2021-01-02 2021-01-24        22
B     2021-01-02 2021-01-24        22
Длительности тестирования одинаковы для всех групп!


### Пропуски

In [49]:
missing_values = data.isnull().sum()
print("Пропущенные значения:")
print(missing_values)
#Удаляем строки с пропусками, если они есть
if missing_values.sum() > 0:
    print("Удаляем строки с пропусками...")
    data = data.dropna()
    print(f"Размер данных после удаления пропусков: {data.shape}")
else:
    print("Пропущенных значений нет")

Пропущенные значения:
user_id     0
date        0
group       0
purchase    0
price       0
dtype: int64
Пропущенных значений нет


### Проверка пользователей в обеих группах

In [50]:
#Находим пользователей, которые были в обеих группах
users_in_both_groups = data.groupby('user_id')['group'].nunique()
users_in_multiple_groups = users_in_both_groups[users_in_both_groups > 1]
print(f"Количество пользователей в обеих группах: {len(users_in_multiple_groups)}")
if len(users_in_multiple_groups) > 0:
    print("Найдены пользователи, которые были в обеих группах!")
    print("Исключаем этих пользователей из анализа...")
    #Получаем ID пользователей для исключения
    users_to_exclude = users_in_multiple_groups.index.tolist()
    #Сохраняем информацию об исключенных пользователях
    excluded_data = data[data['user_id'].isin(users_to_exclude)]
    print(f"Будет исключено {len(excluded_data)} записей")
    #Исключаем пользователей
    data = data[~data['user_id'].isin(users_to_exclude)]
    print(f"Размер данных после исключения: {data.shape}")
else:
    print("Пользователей в обеих группах не найдено!")

Количество пользователей в обеих группах: 1895
Найдены пользователи, которые были в обеих группах!
Исключаем этих пользователей из анализа...
Будет исключено 3790 записей
Размер данных после исключения: (290688, 5)


### Цены

In [51]:
print("\nРаспределение по группам:")
print(data['group'].value_counts())
print("\nСтатистики по покупкам:")
print(data['purchase'].value_counts())
print(f"Конверсия: {data['purchase'].mean():.4f}")
print("\nСтатистики по ценам:")
print(data['price'].describe())
print(f"Средний чек: {data[data['purchase']]['price'].mean():.2f}")


Распределение по группам:
group
B    145381
A    145307
Name: count, dtype: int64

Статистики по покупкам:
purchase
False    255911
True      34777
Name: count, dtype: int64
Конверсия: 0.1196

Статистики по ценам:
count    290688.000000
mean      10151.055427
std       35152.835088
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      200000.000000
Name: price, dtype: float64
Средний чек: 84848.89


## Первичный анализ результатов a/b тестов

In [52]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Вспомогательные харакетристики по группам

In [53]:
group_stats = data.groupby('group').agg(
    visits=('user_id', 'count'),  #количество посещений
    purchases=('purchase', 'sum'),  #количество покупок
    total_revenue=('price', 'sum')  #сумма всех покупок
).reset_index()

In [54]:
group_stats['conversion_rate'] = group_stats['purchases'] / group_stats['visits']
group_stats['average_order_value'] = group_stats['total_revenue'] / group_stats['purchases']

print(group_stats)
print("\n")

  group  visits  purchases  total_revenue  conversion_rate   
0     A  145307      17487     1417780000         0.120345  \
1     B  145381      17290     1533010000         0.118929   

   average_order_value  
0         81076.228055  
1         88664.545980  




### Промежуточные выводы

In [55]:
print("Сопоставимость количества посещений:")
visit_ratio = group_stats[group_stats['group'] == 'B']['visits'].values[0] / group_stats[group_stats['group'] == 'A']['visits'].values[0]
print(f"Соотношение посещений (B/A): {visit_ratio:.3f}")
if 0.9 <= visit_ratio <= 1.1:
    print("Количество посещений сопоставимо (разница менее 10%)")
else:
    print("Количество посещений существенно отличается")
print("\nБалансировка выборок:")
total_visits = group_stats['visits'].sum()
for group in ['A', 'B']:
    group_share = group_stats[group_stats['group'] == group]['visits'].values[0] / total_visits
    print(f"   Группа {group}: {group_share:.1%} посещений")
if 0.45 <= group_stats[group_stats['group'] == 'A']['visits'].values[0] / total_visits <= 0.55:
    print("Выборки сбалансированы")
else:
    print("Выборки несбалансированы")

Сопоставимость количества посещений:
Соотношение посещений (B/A): 1.001
Количество посещений сопоставимо (разница менее 10%)

Балансировка выборок:
   Группа A: 50.0% посещений
   Группа B: 50.0% посещений
Выборки сбалансированы


### Ключевые характеристики и первичные выводы

In [56]:
print("Ключевые метрики по группам:")
key_metrics = group_stats[['group', 'conversion_rate', 'average_order_value', 'total_revenue']].copy()
key_metrics['conversion_rate'] = key_metrics['conversion_rate'].round(4)
key_metrics['average_order_value'] = key_metrics['average_order_value'].round(2)
print(key_metrics)
#Сравнение показателей
conv_A = group_stats[group_stats['group'] == 'A']['conversion_rate'].values[0]
conv_B = group_stats[group_stats['group'] == 'B']['conversion_rate'].values[0]
aov_A = group_stats[group_stats['group'] == 'A']['average_order_value'].values[0]
aov_B = group_stats[group_stats['group'] == 'B']['average_order_value'].values[0]
conv_diff = ((conv_B - conv_A) / conv_A) * 100
aov_diff = ((aov_B - aov_A) / aov_A) * 100
print(f"\nСравнение конверсии:")
print(f"   Группа A: {conv_A:.4f}")
print(f"   Группа B: {conv_B:.4f}")
print(f"   Разница: {conv_diff:+.1f}%")
print(f"\nСравнение среднего чека:")
print(f"   Группа A: {aov_A:.2f}")
print(f"   Группа B: {aov_B:.2f}")
print(f"   Разница: {aov_diff:+.1f}%")
#Первичные выводы
print(f"\nВыводы:")
if conv_B > conv_A:
    print("Тестовая группа B показывает более высокую конверсию")
else:
    print("Контрольная группа A показывает более высокую конверсию")
if aov_B > aov_A:
    print("Тестовая группа B показывает более высокий средний чек")
else:
    print("Контрольная группа A показывает более высокий средний чек")

Ключевые метрики по группам:
  group  conversion_rate  average_order_value  total_revenue
0     A           0.1203             81076.23     1417780000
1     B           0.1189             88664.55     1533010000

Сравнение конверсии:
   Группа A: 0.1203
   Группа B: 0.1189
   Разница: -1.2%

Сравнение среднего чека:
   Группа A: 81076.23
   Группа B: 88664.55
   Разница: +9.4%

Выводы:
Контрольная группа A показывает более высокую конверсию
Тестовая группа B показывает более высокий средний чек
